# Testing the Fine-tuned Math Model

This notebook loads and tests the fine-tuned model from HuggingFace.

In [ ]:
%%capture
!pip install transformers torch accelerate bitsandbytes sympy peft

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import sympy
from sympy.parsing.sympy_parser import parse_expr, standard_transformations, implicit_multiplication_application

# Configure sympy parsing
transformations = standard_transformations + (implicit_multiplication_application,)
x = sympy.Symbol('x')

In [ ]:
# Model configuration
BASE_MODEL = "HuggingFaceTB/SmolLM2-1.7B-Instruct"  # Base model
ADAPTER_MODEL = "Joash2024/Math-SmolLM2-1.7B"       # LoRA adapter
device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

# Load base model and tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

# Load LoRA adapter
print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(model, ADAPTER_MODEL)

In [ ]:
def format_prompt(function: str) -> str:
    """Format input prompt for the model"""
    return f"""Given a mathematical function, find its derivative.

Function: {function}
The derivative of this function is:"""

def generate_derivative(function: str, max_length: int = 200) -> str:
    """Generate derivative for a given function"""
    # Format the prompt
    prompt = format_prompt(function)
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode and extract derivative
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    derivative = generated[len(prompt):].strip()
    
    return derivative

def test_model():
    """Test the model on various functions"""
    test_cases = [
        "x^2",
        "\\sin{\\left(x\\right)}",
        "e^x",
        "\\frac{1}{x}",
        "x^3 + 2x",
        "\\cos{\\left(x^2\\right)}",
        "\\log{\\left(x\\right)}",
        "x e^{-x}"
    ]
    
    print("Testing model on various functions:")
    for func in test_cases:
        print(f"\nFunction: {func}")
        derivative = generate_derivative(func)
        print(f"Generated derivative: {derivative}")

In [ ]:
# Run batch tests
print("Running batch tests...\n")
test_model()

In [ ]:
def interactive_test():
    """Test the model interactively"""
    while True:
        function = input("\nEnter a function (or 'q' to quit): ")
        if function.lower() == 'q':
            break
            
        print("\nGenerating derivative...")
        derivative = generate_derivative(function)
        print(f"Generated derivative: {derivative}")

print("Starting interactive testing...")
print("Enter functions in LaTeX notation (e.g., x^2, \\sin{\\left(x\\right)})")
print("Enter 'q' to quit")
interactive_test()